### SQL2Circuits development notebook

In [1]:
import os
import json
import numpy as np
from data_preparation.queries import QueryGenerator
from data_preparation.prepare import DataPreparation
from data_preparation.database import Database
from circuit_preparation.circuits import Circuits
from training.train import SQL2CircuitsEstimator
from training.sample_feature_preparation import SampleFeaturePreparator
from skopt import BayesSearchCV
from skopt.space import Real
import jax
jax.config.update("jax_enable_x64", True)

this_folder = os.path.abspath(os.getcwd())
seed_paths = ["data_preparation//query_seeds//JOB_query_seed_execution_time.json",
              "data_preparation//query_seeds//JOB_query_seed_cardinality.json"]
workload_types = ["execution_time", "cardinality"]
run_id = 1
ty = 1
workload_type = workload_types[ty]
database = Database("IMDB")
generator = QueryGenerator(run_id, workload_type = "cardinality", database = "IMDB", query_seed_file_path = seed_paths[ty])
query_file = generator.get_query_file()
data_preparator = DataPreparation(run_id, query_file, database = database, workload_type = workload_type, classification = 2)


Number of training queries is  398
Number of test queries is  140
Number of validation queries is  135
cardinality
Error while fetching data from PostgreSQL connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



In [2]:
this_folder = os.path.abspath(os.getcwd())
output_folder = this_folder + "//circuit_preparation//data//circuits//" + str(run_id) + "//"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print("The new directory: ", output_folder, " is created.")

circuits = Circuits(run_id, query_file, output_folder, write_cfg_to_file = True, write_pregroup_to_file=True, generate_circuit_png_diagrams = True)
circuits.execute_full_transformation()

In [3]:
optimization_method = "Pennylane"
sf = SampleFeaturePreparator(run_id, data_preparator, circuits, "all", optimization_method)
X_train = sf.get_X_train()
X_valid = sf.get_X_valid()
y = sf.get_y()

opt = BayesSearchCV(
    SQL2CircuitsEstimator(run_id,
                        circuits = circuits,
                        workload = "cardinality",
                        classification = 2,
                        a = 0.01,
                        c = 0.01,
                        optimization_method = optimization_method,
                        epochs = 500), 
                        { 'a': Real(0.0001, 0.1, 'uniform'), 
                         'c': Real(0.0001, 0.1, 'uniform') }, 
                         n_iter = 2)

opt.fit(X_train, y, X_valid = X_valid)

results = dict(opt.cv_results_)
for key, value in results.items():
    if isinstance(value, np.ndarray):
        results[key] = value.tolist()
best_params = dict(opt.best_params_)
for key, value in best_params.items():
    if isinstance(value, np.ndarray):
        best_params[key] = value.tolist()
results["best_params"] = best_params
with open("training//results//" + str(run_id) + "//" + str(run_id) + "_cv_results_.json", "w") as f:
    json.dump(results, f)

id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane

id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane



Initializing new parameters
273
Number of training circuits:  309
Number of current validation circuits:  126


id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane

iters: 20
train/loss: 1284.8425
train/acc: 0.2524
valid/loss: 471.4543
valid/acc: 0.2778

iters: 40
train/loss: 1165.6837
train/acc: 0.2816
valid/loss: 538.8049
valid/acc: 0.1746

iters: 60
train/loss: 1398.8204
train/acc: 0.2524
valid/loss: 507.72
valid/acc: 0.2222

iters: 80
train/loss: 1227.8535
train/acc: 0.246
valid/loss: 445.5502
valid/acc: 0.3175

iters: 100
train/loss: 1357.3739
train/acc: 0.2589
valid/loss: 512.9008
valid/acc: 0.2143

iters: 120
train/loss: 1279.6617
train/acc: 0.2816
valid/loss: 554.3474
valid/acc: 0.1508

iters: 140
train/loss: 1238.2151
train/acc: 0.233
valid/loss: 486.9967
valid/acc: 0.254

iters: 160
train/loss: 1284.8425
train/acc: 0.288
valid/loss: 518.0816
valid/acc: 0.2063

iters: 180
train/loss: 1258.9384
train/acc: 0.2654
valid/loss: 502.5392
valid/acc: 0.2302

iters: 200
train/loss: 1191.5878
train/acc: 0.2913
valid/loss: 549.1665
valid/acc: 0

Number of circuits:  78 Number of accepted circuits:  78
Score cost:  347.11470276885274
Initializing new parameters
273
Number of training circuits:  309
Number of current validation circuits:  126


id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane

iters: 20
train/loss: 1310.7466
train/acc: 0.2233
valid/loss: 528.4433
valid/acc: 0.1905

iters: 40
train/loss: 1290.0233
train/acc: 0.2104
valid/loss: 507.72
valid/acc: 0.2222

iters: 60
train/loss: 1258.9384
train/acc: 0.2201
valid/loss: 533.6241
valid/acc: 0.1825

iters: 80
train/loss: 1269.3
train/acc: 0.2265
valid/loss: 481.8159
valid/acc: 0.2619

iters: 100
train/loss: 1264.1192
train/acc: 0.2136
valid/loss: 595.7939
valid/acc: 0.0873

iters: 120
train/loss: 1300.3849
train/acc: 0.1909
valid/loss: 471.4543
valid/acc: 0.2778

iters: 140
train/loss: 1321.1082
train/acc: 0.1942
valid/loss: 554.3474
valid/acc: 0.1508

iters: 160
train/loss: 1253.7576
train/acc: 0.2071
valid/loss: 538.8049
valid/acc: 0.1746

iters: 180
train/loss: 1336.6506
train/acc: 0.1845
valid/loss: 518.0816
valid/acc: 0.2063

iters: 200
train/loss: 1300.3849
train/acc: 0.2136
valid/loss: 466.2735
valid/acc: 

...

Number of circuits:  77 Number of accepted circuits:  49
Y_hat:  49 y:  77
Error in score
Initializing new parameters
273
Number of training circuits:  310
Number of current validation circuits:  125


id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane

iters: 20
train/loss: 1248.5768
train/acc: 0.2484
valid/loss: 626.8788
valid/acc: 0.032

iters: 40
train/loss: 1284.8425
train/acc: 0.2387
valid/loss: 492.1776
valid/acc: 0.24

iters: 60
train/loss: 1274.4808
train/acc: 0.2516
valid/loss: 461.0927
valid/acc: 0.288

iters: 80
train/loss: 1227.8535
train/acc: 0.2677
valid/loss: 518.0816
valid/acc: 0.2

iters: 100
train/loss: 1290.0233
train/acc: 0.2161
valid/loss: 461.0927
valid/acc: 0.288

iters: 120
train/loss: 1264.1192
train/acc: 0.229
valid/loss: 492.1776
valid/acc: 0.24

iters: 140
train/loss: 1222.6727
train/acc: 0.2548
valid/loss: 471.4543
valid/acc: 0.272

iters: 160
train/loss: 1290.0233
train/acc: 0.2742
valid/loss: 486.9967
valid/acc: 0.248

iters: 180
train/loss: 1233.0343
train/acc: 0.2871
valid/loss: 616.5172
valid/acc: 0.048

iters: 200
train/loss: 1207.1302
train/acc: 0.2258
valid/loss: 455.9118
valid/acc: 0.296

it

Number of circuits:  77 Number of accepted circuits:  59
Y_hat:  59 y:  77
Error in score
Initializing new parameters
273
Number of training circuits:  309
Number of current validation circuits:  126


id: 1
a: 0.01
c: 0.01
epochs: 500
classification: 4
workload: cardinality
optimization_medthod: Pennylane

iters: 20
train/loss: 1419.5437
train/acc: 0.2524
valid/loss: 486.9967
valid/acc: 0.254

iters: 40
train/loss: 1326.289
train/acc: 0.2589
valid/loss: 549.1665
valid/acc: 0.1587

iters: 60
train/loss: 1176.0453
train/acc: 0.2816
valid/loss: 549.1665
valid/acc: 0.1587

iters: 80
train/loss: 1321.1082
train/acc: 0.2718
valid/loss: 538.8049
valid/acc: 0.1746

iters: 100
train/loss: 1300.3849
train/acc: 0.2686
valid/loss: 543.9857
valid/acc: 0.1667

iters: 120
train/loss: 1119.0564
train/acc: 0.2718
valid/loss: 543.9857
valid/acc: 0.1667

iters: 140
train/loss: 1264.1192
train/acc: 0.2751
valid/loss: 507.72
valid/acc: 0.2222

iters: 160
train/loss: 1284.8425
train/acc: 0.2718
valid/loss: 523.2625
valid/acc: 0.1984

iters: 180
train/loss: 1176.0453
train/acc: 0.2913
valid/loss: 518.0816
valid/acc: 0.2063

iters: 200
train/loss: 1212.3111
train/acc: 0.2654
valid/loss: 533.6241
valid/acc:

: 

In [ ]:
initial_number_of_circuits = 20
number_of_circuits_to_add = 20
total_number_of_circuits = len(data_preparator.get_training_data_labels())

trainer = SQL2CircuitsEstimator(run_id, 
                              workload = "cardinality", 
                              a = 0.001, 
                              c = 0.001, 
                              classification = 2, 
                              optimization_method = "Pennylane")

#for i in range(initial_number_of_circuits, total_number_of_circuits, number_of_circuits_to_add):
#    sf = SampleFeaturePreparator(run_id, data_preparator, circuits, i, "pennylane")
#    X_train = sf.get_X_train()
#    X_valid = sf.get_X_valid()
#    y = sf.get_y()

    #trainer.fit_with_lambeq_noisyopt(X_train, y, X_valid = X_valid, save_parameters = True)
#    trainer.fit_with_pennylane_noisyopt(X_train, y, X_valid, save_parameters = True)
    #trainer.evaluate

# Train for the last time with all the data
sf = SampleFeaturePreparator(run_id, data_preparator, circuits, "all", "pennylane")
X_train = sf.get_X_train()
X_valid = sf.get_X_valid()
y = sf.get_y()

trainer.fit_with_pennylane_noisyopt(X_train, y, X_valid = X_valid, save_parameters = True)


In [1]:
import pennylane as qml
from pennylane import numpy as np

def non_circuit_cost_function(params):
    # Example non-circuit cost function
    return np.sin(params[0]) + params[1]**2

# Define the optimization problem
num_params = 2
initial_params = np.array([0.5, 1.0])
opt = qml.GradientDescentOptimizer(stepsize=0.1)

# Optimization loop
params = initial_params
for _ in range(100):
    params = opt.step(non_circuit_cost_function, params)

print("Optimized parameters:", params)
print("Optimized cost:", non_circuit_cost_function(params))


Optimized parameters: [-1.57069993e+00  2.03703598e-10]
Optimized cost: -0.9999999953538856


In [11]:
import pennylane as qml
import numpy as np

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit():
    qml.PauliZ(wires=0)
    qml.PauliY(wires=1)
    qml.Hadamard(wires=1)
    return qml.density_matrix(wires=[0,1])

# Map the complex values into floats
def real(x):
    return [x.real, x.imag]
print(circuit())
res = [x.real for x in np.array(circuit()).flatten()]
res

[[ 0.5+0.j -0.5+0.j  0. +0.j  0. +0.j]
 [-0.5+0.j  0.5+0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  0. +0.j  0. +0.j]]


[0.4999999999999999,
 -0.4999999999999999,
 0.0,
 0.0,
 -0.4999999999999999,
 0.4999999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [8]:
import pickle
import numpy as np
import json
import os

parameters = "training/results/"
# Process all the folders in the results folder
for folder in os.listdir(parameters):
    folder = parameters + folder + "//"
    if os.path.isdir(folder):
        # Process the pkl file which has the highest number in the beginning
        max_number = 0
        for file in os.listdir(folder):
            if file.endswith(".pkl"):
                number = int(file.split("_")[0])
                if number > max_number:
                    max_number = number
        # Load the parameters and save them to json
        if max_number > 0:
            with open(folder + str(max_number) + "_optax_results_.pkl", 'rb') as f:
                params = pickle.load(f)
            params = np.array(params)
            params = params.tolist()
            with open(folder + str(max_number) + "_parameters.json", 'w') as f:
                json.dump(params, f)